# Loading Packages

In [1]:
using Pkg
path2CloudCovFix = "/Users/baj/Documents/Roman/PIT/prism/hostlight_subtraction/CloudCovFix.jl/"
Pkg.activate(path2CloudCovFix)#*"examples/CloudCovFixEx")

  Activating project at `~/Documents/Roman/PIT/prism/hostlight_subtraction/CloudCovFix.jl`


In [2]:
println("Using the main package now. Not the example project.toml.")

Using the main package now. Not the example project.toml.


In [3]:
# Pkg.instantiate()

In [4]:
# path to our simulated data
path2simdata = "/Users/baj/Documents/Roman/PIT/prism/hostlight_subtraction/sf22_romanprism/prismdata_notebook/"

"/Users/baj/Documents/Roman/PIT/prism/hostlight_subtraction/sf22_romanprism/prismdata_notebook/"

In [5]:
# the first time you run this, it might take ~5 min, but it will be faster in the future
# and for all other julia environments you install (lots of package caching and precompilation)
first_time = false
if first_time
    Pkg.add(url="https://github.com/andrew-saydjari/CloudCovFix.jl")
    Pkg.add("FITSIO")
    Pkg.add(["CairoMakie","ColorSchemes"])
end

In [6]:
# just bear with it for the first time extra precompilation times (~1-2 min)
using CloudCovFix, FITSIO, StatsBase

In [7]:
# Pkg.add("CairoMakie")

In [8]:
# Pkg.add("ColorSchemes")

In [9]:
# just bear with it for the first time extra precompilation times (~3-4 min)
using CairoMakie; set_theme!(theme_black()); CairoMakie.disable_mime!("svg", "pdf", "text/html");
using ColorSchemes

# Synthetic Filament Example

Here we show CloudCovFix performing well on the difficult example of infilling a filament that is isolated on a flat background, in the presence of noise. It is of note that the infill is correctly anisotropic, in contrast to most kernel methods that make an isotropic assumption.

In [10]:
f = FITS(path2simdata*"Roman_TDS_truth_SNPrism_0_5_1000_galsn.fits")
out_image = read(f[1])
close(f)

In [11]:
# Create a rectangular mask
xsn = 2036.7435
ysn = 2046.6222

xsize = 12  # pixels along x; cross-dispersion dir
ysize = 220  # pixels along y; dispersion dir

rectmask = falses(xsize, ysize)

# set the central area with SN spectrum to be masked
rectmask[3:10, 10:210] .= true


12×220 BitMatrix:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1     1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1     1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1     1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1  …  1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1     1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1     1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1     1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  1  1  1  1     1  1  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0

In [12]:
# Save the rectangular mask to check
# first check for old mask file. delete if it exists
maskfile = "rectmask.fits"
if isfile(maskfile)
    rm(maskfile)
    println("Deleted old mask file.")
end

maskfits = FITS(maskfile, "w")
write(maskfits, Int.(rectmask))
close(maskfits)
println("Saved new mask.")

Saved new mask.


In [ ]:

Np = 25  # has to be odd?
halfNp = (Np-1)÷2
dv = halfNp
shiftx = 0

bimage = zeros(Bool,size(out_image));

# This line will create a mask matrix of size Np x Np
# where values outside radius rp are set to zero.
# i.e., the pixels inside the radius are masked.
circmask = .!kstar_circle_mask(Np,rlim=4^2);

cenx = 2036
ceny = 2036

bimage[(cenx+shiftx-dv):(cenx+shiftx+dv),(ceny-dv):(ceny+dv)] .= circmask;
out_image[bimage] .= 0

count(bimage)


In [15]:
cenx = round(Int64, xsn)
ceny = round(Int64, ysn)

println("Masking SN spectrum with location (approx int pix):")
println("x: ", cenx, "   y: ", ceny)

dx = div(xsize, 2)
dy = div(ysize, 2)

println("dx: ", dx, "    dy: ", dy)

Np = 25

bimage = zeros(Bool, size(out_image));
bimage[(cenx-dx):(cenx+dx-1), (ceny-dy):(ceny+dy-1)] .= rectmask;
out_image[bimage] .= 0
count(bimage)

Masking SN spectrum with location (approx int pix):
x: 2037   y: 2047
dx: 6    dy: 110


1608

In [16]:
ndraw0 = 100
widx = 5

x_locs = [cenx]
y_locs = [ceny]
star_stats = proc_discrete(x_locs,y_locs,out_image,bimage,Np=Np,tilex=4,ftype=64,widx=widx,seed=2022,ndraw=ndraw0);

BAJ EDIT: Using edited version with initial infilling width widx=5
BAJ EDIT: No need for Pkg.add or Pkg.instantiate now. The main CloudCovFix pkg and all deps have been added.
BAJ EDIT: Now simply edit the code as needed.
BAJ EDIT: Every time you edit you will need to restart kernel and run the initial lines.
BAJ EDIT: Remove these lines once done.
Infilling completed after 4 rounds with final width (widx,widy) = (19,19)
Finished 0 stars in tile (1, 1)
Finished 0 stars in tile (1, 2)
Finished 0 stars in tile (1, 3)
Finished 0 stars in tile (1, 4)
Finished 0 stars in tile (2, 1)
Finished 0 stars in tile (2, 2)
Had to use SVD for icov_kgood_kgood at (x,y) = (2037,2047)
Had to use SVD for sqrt_cov at (x,y) = (2037,2047)
Finished 1 stars in tile (2, 3)
Finished 0 stars in tile (2, 4)
Finished 0 stars in tile (3, 1)
Finished 0 stars in tile (3, 2)
Finished 0 stars in tile (3, 3)
Finished 0 stars in tile (3, 4)
Finished 0 stars in tile (4, 1)
Finished 0 stars in tile (4, 2)
Finished 0 stars 

In [17]:
# Save the new infilled array to FITS
newfits = FITS("Roman_TDS_truth_SNPrism_0_5_1000_galsn_infilled.fits", "w");
write(newfits, star_stats[1])
close(newfits)

In [ ]:
# Create a difference image and save that
diffarr = out_image - star_stats[1]
# Now save
difffits = FITS("Roman_TDS_truth_SNPrism_0_5_1000_galsn_diff.fits", "w");
write(difffits, diffarr)
close(difffits)

In [ ]:
dv = 100
dummy = 10  # added to make everything positive. not sure how to sub -ve with nan yet
dataarr = star_stats[1][(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv)]'.+dummy

In [ ]:

fig = Figure(size=(600, 600), fontsize=18)
ax = Axis(fig[1,1], aspect=1, title="Mean Infill", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)
data = log10.(dataarr)
h = heatmap!(ax, data, colormap=:thermal, colorrange=(1, 3))

resize_to_layout!(fig)
fig

In [ ]:
dv = 50
fig = Figure(size=(600, 600), fontsize=18)
ax = Axis(fig[1,1], aspect=1, title="Draw Infill", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)
h = heatmap!(ax,star_stats[2][(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv),1]'.-100,
    colormap=:diverging_bkr_55_10_c35_n256,
    colorrange=(-50, 50)
)

resize_to_layout!(fig)
fig

# WISE Infilling Example

Example of using proc_continuous to infill an arbitrary masked region of an image. This examples uses a subimage from the WISE 12 um dust map (tile 115).

In [ ]:
f = FITS(path2CloudCovFix*"examples/data/WISE_115sub.fits")
raw_image = read(f[1])
close(f)
out_image = copy(raw_image);

In [ ]:
Np = 33
halfNp = (Np-1)÷2
dv = halfNp
shiftx = 5
bimage = zeros(Bool,size(out_image));
circmask = .!kstar_circle_mask(33,rlim=6^2);
cenx = 579
ceny = 579
bimage[(cenx+shiftx-dv):(cenx+shiftx+dv),(ceny-dv):(ceny+dv)].=circmask;
cenx = 550
ceny = 550
bimage[(cenx+shiftx-dv):(cenx+shiftx+dv),(ceny-dv):(ceny+dv)].=circmask;
out_image[bimage].=0
count(bimage)

In [ ]:
star_stats = proc_continuous(out_image,bimage,Np=Np,tilex=4,ftype=64,widx=widx,seed=2022,ndraw=ndraw0);

In [ ]:
cenx = 579
ceny = 579
dv = 50
vmin, vmax = percentile(raw_image[(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv)],[2,99])
fig = Figure(size=(600, 600), fontsize=18)
ax = Axis(fig[1,1], aspect=1, title="Mean Infill", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)
h = heatmap!(ax,star_stats[1][(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv)]',
    colormap=:rainbow_bgyr_35_85_c73_n256,
    colorrange=(vmin,vmax)
)

resize_to_layout!(fig)
fig

In [ ]:
dv = 50
fig = Figure(size=(600, 600), fontsize=18)
ax = Axis(fig[1,1], aspect=1, title="Draw Infill", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)
h = heatmap!(ax,star_stats[2][(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv),1]',
    colormap=:rainbow_bgyr_35_85_c73_n256,
    colorrange=(vmin,vmax)
)

resize_to_layout!(fig)
fig

In [ ]:
dv = 50
fig = Figure(size=(600, 600), fontsize=18)
ax = Axis(fig[1,1], aspect=1, title="Original Image", xticksvisible=false, yticksvisible=false, 
          xticklabelsvisible=false, yticklabelsvisible=false)
h = heatmap!(ax,raw_image[(cenx-dv):(cenx+dv),(ceny-dv):(ceny+dv)]',
    colormap=:rainbow_bgyr_35_85_c73_n256,
    colorrange=(vmin,vmax)
)

resize_to_layout!(fig)
fig